In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime 

## Web Scrapping IMSSS

### Web Scrapping de la página general por años

In [38]:
# Guardamos el url base para poder concatenar los links de los archivos
base_url = "http://datos.imss.gob.mx"
# Obtenemos el contenido de la página
r = requests.get("http://datos.imss.gob.mx/dataset")
# Creamos el objeto soup
soup = BeautifulSoup(r.content, "html.parser") 
# Obtenemos todos los links de la página
spans = soup.find_all("a", {'data-format':'csv'}, href=True)
# Se guardaran los links de los archivos que nos interesan por año
links = []
# Obtenemos los links de los archivos que nos interesan
for link in spans:
    href = link.get("href")
    if '/dataset/asg-' in href:
        condicion = int(re.findall(r'\d{4}', href)[0])
        if condicion >= 2018 and condicion < 2023:
            links.append(base_url + href)

#### Descripción de fuentes

In [49]:
# Crea y abre el archivo donde se excribe la información de la referncia
file = open('descripciones\imss_descripcion.txt', 'w+',encoding='utf-8')
file.write('Datos del IMSS\n')
for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    file.write('Enlace de archivos por mes: '+link+'\n')
    # Tabla de referencias
    table = soup.find('table', class_= 'field-group-format group_additional')
    # Moviendose por filas
    rows = table.find_all('tr')
    for row in rows[1:]:
        if row:
            cells = row.find_all(['th', 'td'])
            # Extrae el texto de las celdas y los concatena
            file.write(': '.join(cell.get_text(strip=True) for cell in cells))
            file.write('\n')
# Agrega la fecha en la que se corrió el script que extrajo los datos
fecha_actual = datetime.now().strftime('%Y-%m-%d')
file.write('\nFecha de descarga: '+fecha_actual+'\n')
file.close()

### Web Scrapping parte 2 por meses

In [4]:
# Por cada año, guardaremos los links de los archivos que nos interesan por meses
links_2 = []
# Obtenemos los links de los archivos que nos interesan
for i in range(len(links)):
    r_2 = requests.get(links[i])
    soup_2 = BeautifulSoup(r_2.content, "html.parser")
    spans_2 = soup_2.find_all("a", {'property':'dcat:accessURL'}, href=True)
    for link in spans_2:
        links_2.append(base_url + link.get("href"))

In [18]:
# Creamos una lista para guardar los dataframes
df = []
# Obtenemos los links de los archivos que nos interesan
for i in range(len(links_2)):
    r_3 = requests.get(links_2[i])
    soup_3 = BeautifulSoup(r_3.content, "html.parser")
    spans_3 = soup_3.find_all("a", href=True)
    for link in spans_3:
        if '.csv' in link.get("href"):
            df.append(link.get("href"))

In [ ]:

# Crear un DataFrame vacío para almacenar los datos combinados
df_combined = pd.DataFrame()

for i in range(len(df)):
    # Leer el archivo CSV y especificar el tipo de datos adecuado (si es necesario)
    df_csv = pd.read_csv(df[i], encoding="latin-1", sep="|", dtype={"columna_con_tipo_mezclado": str}, low_memory=False)

    columns = ['rango_uma', 'ta', 'teu', 'tec', 'tpu', 'tpc',
               'ta_sal', 'teu_sal', 'tec_sal', 'tpu_sal', 'tpc_sal', 'masa_sal_ta',
               'masa_sal_teu', 'masa_sal_tec', 'masa_sal_tpu', 'masa_sal_tpc']

    # Usar copy() para evitar SettingWithCopyWarning
    df_sonora = df_csv.loc[(df_csv["sector_economico_1"] == 0) & (df_csv["cve_entidad"] == 26)].copy()

    # Eliminar columnas
    df_sonora.drop(columns, inplace=True, axis=1)

    # Extraer año y mes de la URL
    df_sonora['año'] = re.findall(r'\d{4}', df[i])[0]
    df_sonora['mes'] = re.findall(r'-(\d{2})-', df[i])[0]

    # Filtrar por sector_economico_2
    df_append = df_sonora[df_sonora["sector_economico_2"].isin([1, 2, 4])]

    # Usar pd.concat en lugar de append (sin ignore_index=True)
    df_combined = pd.concat([df_combined, df_append])

In [ ]:
# from google.colab import files
df_combined.to_csv('data\df_imss.csv') 
# files.download('df_imss.csv')

### Diccionario y catálogo

In [5]:
dic_cat = pd.read_excel('http://datos.imss.gob.mx/sites/default/files/diccionario_de_datos_1.xlsx', sheet_name=None)

for sheet_name, sheet_data in dic_cat.items():
    if sheet_name == 'ejemplo archivo':
        continue  # Ignorar esta hoja y pasar a la siguiente
    elif sheet_name == 'Layout':
        # Guarda el layout en un archivo de Excel
        sheet_data = dic_cat['Layout']
        sheet_data.to_excel('diccionario_IMSS.xlsx')       
    else:
        # Reemplaza los espacios en el nombre de la hoja con guiones bajos
        nombre_archivo_csv = f"imss_{sheet_name.replace(' ', '_')}.csv"
        # Guarda el archivo CSV en el directorio 'data'
        sheet_data.to_csv(f'data/{nombre_archivo_csv}')
        print(f"La hoja '{sheet_name}' se ha guardado en '{nombre_archivo_csv}'")


entidad-municipio
sector 2
delegación-subdelegación
ejemplo archivo
Layout
sector 4
La hoja 'delegación-subdelegación' se ha guardado en 'imss_delegación-subdelegación.csv'
La hoja 'entidad-municipio' se ha guardado en 'imss_entidad-municipio.csv'
La hoja 'sector 1' se ha guardado en 'imss_sector_1.csv'
La hoja 'sector 2' se ha guardado en 'imss_sector_2.csv'
La hoja 'sector 4' se ha guardado en 'imss_sector_4.csv'
La hoja 'Tamaño de registro patronal' se ha guardado en 'imss_Tamaño_de_registro_patronal.csv'
La hoja 'sexo' se ha guardado en 'imss_sexo.csv'
La hoja 'Rango edad' se ha guardado en 'imss_Rango_edad.csv'
La hoja 'Rango salario' se ha guardado en 'imss_Rango_salario.csv'
La hoja 'Rango UMA' se ha guardado en 'imss_Rango_UMA.csv'


## Función para catálogos

In [2]:
def catalogos(tipo_catalogo, soup):
    # Utilizar una expresión regular para buscar elementos <a> con 'title' que contiene 'Catalogos'
    pattern = re.compile(r'Cat[aá]logos', re.IGNORECASE)  # La opción re.IGNORECASE hace que la búsqueda sea insensible a mayúsculas/minúsculas
    enlace = soup.find_all("a", {'title':pattern}, href=True)
    for link in enlace:
        href = link.get("href")
        df = pd.read_excel(href, sheet_name=None)
        # Iteramos a través de las hojas del archivo
        for sheet_name, sheet_data in df.items():
            # Genera un nombre de archivo CSV basado en el nombre de la hoja
            nombre_archivo_csv = f"{tipo_catalogo}_{sheet_name}.csv"
            # Guarda el DataFrame de la hoja en un archivo CSV
            sheet_data.to_csv(nombre_archivo_csv, index=False)
            print(f"La hoja '{sheet_name}' se ha guardado en '{nombre_archivo_csv}'")

## Web Scraping Agricultura (2018-2022)

In [73]:
# Guardamos el url
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-de-agricultura-sonora/1573'
# Obtenemos el contenido de la página
r = requests.get(url)
# Creamos el objeto soup
soup = BeautifulSoup(r.content, "html.parser")
# Obtenemos todos los links de la página
spans = soup.find_all("a", href=True)

# Información de la fuente
file = open('descripciones\\agricultura_descripcion.txt', 'w+', encoding = 'utf-8')
file.write('Enlace de los archivos: '+url+'\n')
data_info = soup.find('p', class_='mb-3').get_text()
info = data_info.split(' | ')
file.write('\n'.join(info))
file.write('\nFecha de descarga: '+fecha_actual)

file.close()

In [ ]:
# Llamamos a la función catalogos
catalogos('Agricultura', soup)

In [ ]:
# Lista de años
years = ['2018', '2019', '2020', '2021', '2022']
links_xlsx = []
# Recorre la lista de años y busca enlaces con títulos que coincidan con cada año
for year in years:
    links = soup.find_all("a", {'title': f'Producción agrícola {year}'}, href=True)
    for link in links:
        links_xlsx.append(link['href'])

# Creamos un DataFrame vacío para almacenar los datos combinados
df_agricultura = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_agricultura = pd.concat([df_agricultura, pd.read_excel(links_xlsx[i])])

# Convertimos el df a csv
df_agricultura.to_csv('data\df_agricultura.csv')

## Web Scraping Pecuaria (2018-2022)

In [74]:
# URL de la página HTML
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-ganaderia-sonora/1581'
# Realiza la solicitud HTTP
r = requests.get(url)
# Parsea el contenido HTML
soup = BeautifulSoup(r.content, "html.parser")

# Información de la fuente
file = open('descripciones\\pecuaria_descripcion.txt', 'w+', encoding = 'utf-8')
file.write('Enlace de los archivos: '+url+'\n')
data_info = soup.find('p', class_='mb-3').get_text()
info = data_info.split(' | ')
file.write('\n'.join(info))
file.write('\nFecha de descarga: '+fecha_actual)

file.close()

In [ ]:
# Llamamos a la función catalogos
catalogos('Pecuaria', soup)

In [ ]:
# Lista de años
years = ['2018', '2019', '2020', '2021', '2022']
# Lista para almacenar los enlaces .xlsx
links_xlsx = []
# Expresión regular para buscar enlaces .xlsx
xlsx_pattern = re.compile(r'\.xlsx$')
# Busca todos los enlaces con extensión .xlsx en la página
enlaces = soup.find_all('a', href=xlsx_pattern)

# Recorre los enlaces y verifica si el texto del enlace contiene un año de la lista
for enlace in enlaces:
    texto_enlace = enlace.text
    for year in years:
        if year in texto_enlace:
            links_xlsx.append(enlace['href'])

# Elimina duplicados (si los hay) utilizando set y luego convierte de nuevo a lista
links_xlsx = list(set(links_xlsx))

# Creamos un DataFrame vacío para almacenar los datos combinados
df_pecuaria = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_pecuaria = pd.concat([df_pecuaria, pd.read_excel(links_xlsx[i])])

# Convertimos el df a csv
df_pecuaria.to_csv('data\df_pecuaria.csv')

## Web Scraping Pesca (2018-2022)

In [75]:
# URL de la página HTML
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-pesca-sonora/1582'

# Realiza la solicitud HTTP
r = requests.get(url)

# Parsea el contenido HTML
soup = BeautifulSoup(r.content, "html.parser")
# Información de la fuente
file = open('descripciones\\pesca_descripcion.txt', 'w+', encoding = 'utf-8')
file.write('Enlace de los archivos: '+url+'\n')
data_info = soup.find('p', class_='mb-3').get_text()
info = data_info.split(' | ')
file.write('\n'.join(info))
file.write('\nFecha de descarga: '+fecha_actual)

file.close()


In [ ]:
# Lista de años
years = ['2018', '2019', '2020', '2021', '2022']
# Lista para almacenar los enlaces .xlsx
links_xlsx = []
# Expresión regular para buscar enlaces .xlsx
xlsx_pattern = re.compile(r'\.xlsx$')
# Busca todos los enlaces con extensión .xlsx en la página
enlaces = soup.find_all('a', href=xlsx_pattern)

# Recorre los enlaces y verifica si el texto del enlace contiene un año de la lista
for enlace in enlaces:
    texto_enlace = enlace.text
    for year in years:
        if year in texto_enlace:
            links_xlsx.append(enlace['href'])

# Elimina duplicados (si los hay) utilizando set y luego convierte de nuevo a lista
links_xlsx = list(set(links_xlsx))

# Creamos un DataFrame vacío para almacenar los datos combinados
df_pesca = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_pesca = pd.concat([df_pesca, pd.read_excel(links_xlsx[i])])

# Convertimos el df a csv
df_pesca.to_csv('data\df_pesca.csv')